# Imports and Libraries

In [126]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

import datetime as dt
# import pandas as pd

# import os
import pickle
import uuid

In [127]:
client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production')

# Get OTPI Sheet

In [128]:
SAMPLE_SPREADSHEET_ID_input = '1lalOwWx_GwUZgjG3WgyxZvirtT9GYDJ6t3dAuNimTLM'
SAMPLE_RANGE_NAME = 'OTPI Scores!A:V'

opti_scores = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)



# Pick Game and Team to generate invoices for

In [129]:
print(opti_scores['Game Date'].drop_duplicates())

0       5 Aug
63     12 Aug
132    19 Aug
212    26 Aug
279    02 Sep
Name: Game Date, dtype: object


In [130]:
game='02 Sep'
teams = ['2', '3','4']



invoices_to_raise = opti_scores[opti_scores['Game Date']==game]
invoices_to_raise = invoices_to_raise[invoices_to_raise['OT Team'].isin(teams)]

invoices_to_raise

,Game Date,Competition,Player Name,Position,OT Team,Opposition Name,Result,Goals,Assists,Goals Conceded,...,Red Cards,Clean Sheet?,Penalties Missed,Penalties Saved,Own Goals,MOTM?,Kit,Amount Owed,OPTI Score,Missing
308,02 Sep,Friendly,Alex Scott-Ruddock,GK,4,Suttonians 4s,Loss,,,5,...,,FALSE,,,,1,,10,None,None
309,02 Sep,Friendly,Kevin Gonzalez,DEF,4,Suttonians 4s,Loss,,,5,...,,FALSE,,,,0,,5,None,None
310,02 Sep,Friendly,Ade Lawal,DEF,4,Suttonians 4s,Loss,,,5,...,,FALSE,,,,0,,10,None,None
311,02 Sep,Friendly,Jack Butcher,DEF,4,Suttonians 4s,Loss,,,5,...,,FALSE,,,,0,,10,None,None
312,02 Sep,Friendly,Cameron Hildebrandt-Brooks,DEF,4,Suttonians 4s,Loss,,,5,...,,FALSE,,,,0,,5,None,None
313,02 Sep,Friendly,Nima Naghdi,MID,4,Suttonians 4s,Loss,,,5,...,,FALSE,,,,0,TRUE,0,None,None
314,02 Sep,Friendly,George Havord,MID,4,Suttonians 4s,Loss,1,,5,...,,FALSE,,,,0,,10,None,None
315,02 Sep,Friendly,Masood Hamidi,MID,4,Suttonians 4s,Loss,1,,5,...,,FALSE,,,,0,,10,None,None
316,02 Sep,Friendly,Leuman Coronado,MID,4,Suttonians 4s,Loss,,,5,...,,FALSE,,,,0,,10,None,None
317,02 Sep,Friendly,Pearl Bakengela,MID,4,Suttonians 4s,Loss,1,,5,...,,FALSE,,,,0,,10,None,None


# Get Current Debt

In [131]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Overall Summary!B2:D1000'

matchday_debt = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)
matchday_debt = matchday_debt[matchday_debt['Name']!='']

matchday_debt

,Name,22-23 Debt,Match Day Debt
0,Aamish Ibrahi,£0.00,£0.00
1,Aaron Snelling,£0.00,£10.00
2,Abdul Adaawe,£0.00,£0.00
3,Abdul Hassan,£0.00,£0.00
4,Adam Platten,£0.00,£5.00
...,...,...,...
308,Will French,£0.00,-£5.00
309,Will Salt,£0.00,£0.00
310,Will Sollory,£0.00,£0.00
311,Yemi Oviosu,£0.00,£20.00


# Merge to Opti

In [132]:
invoice_mg_1 = pd.merge(left=invoices_to_raise, right = matchday_debt, how = 'left', right_on='Name', left_on = 'Player Name')
invoice_mg_1['Match Day Debt'] = pd.to_numeric(invoice_mg_1['Match Day Debt'].str.replace('£', ''))
invoice_mg = invoice_mg_1[invoice_mg_1['Match Day Debt']>0]
#
invoice_mg=invoice_mg[['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']]

print('invoices to be made')
display(invoice_mg)
print(invoice_mg.shape)

print('up to date')
display(invoice_mg_1[invoice_mg_1['Match Day Debt']<=0][['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']])
print(invoice_mg_1[invoice_mg_1['Match Day Debt']<=0][['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']].shape)

invoices to be made


,Player Name,OT Team,Game Date,Opposition Name,Amount Owed
2,Ade Lawal,4,02 Sep,Suttonians 4s,10
8,Leuman Coronado,4,02 Sep,Suttonians 4s,10
12,Christian King,4,02 Sep,Suttonians 4s,10
15,Joey Wackrill,2,02 Sep,Old Tenisonians 2s,10
16,Finn Coolen,2,02 Sep,Old Tenisonians 2s,5
17,Sam Laidler,2,02 Sep,Old Tenisonians 2s,5
19,Corey Cleave,2,02 Sep,Old Tenisonians 2s,10
20,Tom Riley,2,02 Sep,Old Tenisonians 2s,10
22,Harold Kiyingi,2,02 Sep,Old Tenisonians 2s,10
24,Joe Docherty,2,02 Sep,Old Tenisonians 2s,10


(16, 5)
up to date


,Player Name,OT Team,Game Date,Opposition Name,Amount Owed
0,Alex Scott-Ruddock,4,02 Sep,Suttonians 4s,10
1,Kevin Gonzalez,4,02 Sep,Suttonians 4s,5
3,Jack Butcher,4,02 Sep,Suttonians 4s,10
4,Cameron Hildebrandt-Brooks,4,02 Sep,Suttonians 4s,5
5,Nima Naghdi,4,02 Sep,Suttonians 4s,0
6,George Havord,4,02 Sep,Suttonians 4s,10
7,Masood Hamidi,4,02 Sep,Suttonians 4s,10
9,Pearl Bakengela,4,02 Sep,Suttonians 4s,10
10,Sam A'Court,4,02 Sep,Suttonians 4s,10
11,Joe France,4,02 Sep,Suttonians 4s,10


(26, 5)


# Get Customer IDs 

In [133]:
customers_api = client.customers

In [137]:

result = customers_api.list_customers(
    
)
customers = {str(cust.get('given_name')) + ' ' + str(cust.get('family_name')):cust.get('id') for cust in result.body['customers']}


i=1
while result.cursor:
    print(i)
    i+=1
    result = customers_api.list_customers(
        cursor = result.cursor
    )
    customers2 = {str(cust.get('given_name')) + ' ' + str(cust.get('family_name')):cust.get('id') for cust in result.body['customers']}

    customers.update(customers2)




1
2
3
4
5
6
7
8


In [138]:
customer_df=pd.DataFrame(data=[{'name':cust, 'id':customers[cust]} for cust in customers])

In [140]:
id_mg = pd.merge(left = invoice_mg, right = customer_df, how= 'left', left_on = 'Player Name', right_on = 'name')

id_mg['Amount Owed']=pd.to_numeric(id_mg['Amount Owed'])



id_mg

,Player Name,OT Team,Game Date,Opposition Name,Amount Owed,name,id
0,Ade Lawal,4,02 Sep,Suttonians 4s,10,Ade Lawal,T57NBJNMRHX79ZV68VW8SK166R
1,Leuman Coronado,4,02 Sep,Suttonians 4s,10,Leuman Coronado,K97T0162XDDT1NMPN9YQGCH2WR
2,Christian King,4,02 Sep,Suttonians 4s,10,Christian King,S7CZGD28C50J12XGC8AWTFG83M
3,Joey Wackrill,2,02 Sep,Old Tenisonians 2s,10,Joey Wackrill,VHEZQ85XHX1GK1ZGPW49PR99Z0
4,Finn Coolen,2,02 Sep,Old Tenisonians 2s,5,Finn Coolen,GX8ZN9VGKTHBW86T2VBBTKZNNM
5,Sam Laidler,2,02 Sep,Old Tenisonians 2s,5,Sam Laidler,Z9YKSNFDZ0JDBTEHGXJS2K1P3G
6,Corey Cleave,2,02 Sep,Old Tenisonians 2s,10,Corey Cleave,0HTAK3MQ28S69A5KBCTZ8HKHC0
7,Tom Riley,2,02 Sep,Old Tenisonians 2s,10,Tom Riley,HETKAQ4MN17GHFSVRBG687N2JG
8,Harold Kiyingi,2,02 Sep,Old Tenisonians 2s,10,Harold Kiyingi,K7PXE48P0H4Z5EW9ZMAZVKN6ZG
9,Joe Docherty,2,02 Sep,Old Tenisonians 2s,10,Joe Docherty,R4RHW9Q96H4C30FRJPTQ8TN25W


# Create Orders

In [141]:
orders_api = client.orders

new_orders = []

for i, row in id_mg.iterrows():
    print(i)
    amount = row['Amount Owed'] 
    body = {
        'order': {
            'location_id': 'SBBJSGR24YHYB',
            'reference_id': f'{uuid.uuid1()}',
            'line_items': [
                
                {
                    'quantity': '1',
                    'catalog_object_id': 'OKAX666RRB7XKNM6BJTRM6IM',
                    'base_price_money': {
                        'amount': amount*100,
                        'currency': 'GBP'
                    }
                }
            ]
        },
        'idempotency_key': f'{uuid.uuid1()}'
    }
    # print(body)
    
    result = orders_api.create_order(body)
    # print(result)
    obj_id=result.body['order']['id']

    new_orders.append({'Player Name':row['Player Name'], 'obj_id':obj_id})
    # # print(result)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [142]:
order_mg= pd.merge(left = id_mg, right=pd.DataFrame(new_orders), how= 'left', on = 'Player Name')

order_mg

,Player Name,OT Team,Game Date,Opposition Name,Amount Owed,name,id,obj_id
0,Ade Lawal,4,02 Sep,Suttonians 4s,10,Ade Lawal,T57NBJNMRHX79ZV68VW8SK166R,tfIQU8O9eVkQgGxvtGhFHnNUY7eZY
1,Leuman Coronado,4,02 Sep,Suttonians 4s,10,Leuman Coronado,K97T0162XDDT1NMPN9YQGCH2WR,9vE8KHYhvsMySMb1UPDxwTCfJHPZY
2,Christian King,4,02 Sep,Suttonians 4s,10,Christian King,S7CZGD28C50J12XGC8AWTFG83M,nIiYZ3kfEB2Rp0H1QtoHOqTjqYXZY
3,Joey Wackrill,2,02 Sep,Old Tenisonians 2s,10,Joey Wackrill,VHEZQ85XHX1GK1ZGPW49PR99Z0,hfVxc5adN9bZPCurJJrAWEVCYWZZY
4,Finn Coolen,2,02 Sep,Old Tenisonians 2s,5,Finn Coolen,GX8ZN9VGKTHBW86T2VBBTKZNNM,1ZZN3XvvPDDqWKr5o5OqGUiTT8SZY
5,Sam Laidler,2,02 Sep,Old Tenisonians 2s,5,Sam Laidler,Z9YKSNFDZ0JDBTEHGXJS2K1P3G,zYNb7Ma2bsuDpmM3Hr7CdZdaddYZY
6,Corey Cleave,2,02 Sep,Old Tenisonians 2s,10,Corey Cleave,0HTAK3MQ28S69A5KBCTZ8HKHC0,5LvCuVXJi4SbtnSTDjWOrXiMBzaZY
7,Tom Riley,2,02 Sep,Old Tenisonians 2s,10,Tom Riley,HETKAQ4MN17GHFSVRBG687N2JG,jOc90auJoXLe0uBQ6KMAGkXC38LZY
8,Harold Kiyingi,2,02 Sep,Old Tenisonians 2s,10,Harold Kiyingi,K7PXE48P0H4Z5EW9ZMAZVKN6ZG,t5DNpFgjAqOrOXgdxf3sWc0LFrOZY
9,Joe Docherty,2,02 Sep,Old Tenisonians 2s,10,Joe Docherty,R4RHW9Q96H4C30FRJPTQ8TN25W,xrd6VU2RY4PPofA72IiDTNygU8YZY


# Create Invoices

In [143]:
invoices_api = client.invoices

In [144]:
invoices = []

for i, row in order_mg.iterrows():
    order_id = row['obj_id']
    cust_id = row['id']
    invoice_number = str(dt.datetime.now().date()) +f"_{row['OT Team']}"+f"_{row['obj_id']}_1"

    due_at =  str(dt.datetime.now().date() + dt.timedelta(days = 1))

    title = 'Match Subs ' + row['Game Date'] + ' vs ' + row['Opposition Name']

        
    body = {
    'invoice': {
        'location_id': 'SBBJSGR24YHYB',
        'order_id': f'{order_id}',
        'primary_recipient': {
            'customer_id': f'{cust_id}'
        },
        'payment_requests': [
            {
                'request_type': 'BALANCE',
                'due_date': due_at,
                'tipping_enabled': True,
                'automatic_payment_source': 'NONE',
                'reminders': [
                    {
                        'relative_scheduled_days': -1,
                        'message': 'Your Invoice is due'
                    }
                ]
            }
        ],
        'delivery_method': 'EMAIL',
        'invoice_number': invoice_number,
        'title': title,
        'description': title,
        'scheduled_at': due_at+'T10:00:00Z',
        'accepted_payment_methods': {
            'card': True,
            'square_gift_card': False,
            'bank_account': False,
            'buy_now_pay_later': False,
            'cash_app_pay': False
        },
        'sale_or_service_date': str(dt.datetime.now().date()),
        'store_payment_method_enabled': False
    },
    'idempotency_key': f'{uuid.uuid1()}'
    }

    result = invoices_api.create_invoice(body)
    print(result)
    invidid=result.body['invoice']['id']
    invoices.append(invidid)

<ApiResponse {"invoice":{"id":"inv:0-ChAImh-csCAng261EGwHqsxAEIMP","version":0,"location_id":"SBBJSGR24YHYB","order_id":"tfIQU8O9eVkQgGxvtGhFHnNUY7eZY","payment_requests":[{"uid":"769ce965-0392-460b-ad10-3f8b7ea8e600","request_type":"BALANCE","due_date":"2023-09-04","tipping_enabled":true,"computed_amount_money":{"amount":1000,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"e05d75bc-d725-487f-b568-2d7bd468c324","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"PENDING"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"T57NBJNMRHX79ZV68VW8SK166R","given_name":"Ade","family_name":"Lawal","email_address":"adelawal@live.co.uk","phone_number":"+447769116015"},"invoice_number":"2023-09-03_4_T57NBJNMRHX79ZV68VW8SK166R_1","title":"Match Subs 02 Sep vs Suttonians 4s","description":"Match Subs 02 Sep vs Suttonians 4s","scheduled_at":"2023-09-04T10:00:00Z","status":"DRAFT","timezone":"Europe/London","

In [145]:
for invidid in invoices:

    body = {
        'version': 0,
        'idempotency_key': f'{invidid}'
    }

    result = invoices_api.publish_invoice(
        invidid,
        body
    )
    print(result)

<ApiResponse {"invoice":{"id":"inv:0-ChAImh-csCAng261EGwHqsxAEIMP","version":1,"location_id":"SBBJSGR24YHYB","order_id":"tfIQU8O9eVkQgGxvtGhFHnNUY7eZY","payment_requests":[{"uid":"769ce965-0392-460b-ad10-3f8b7ea8e600","request_type":"BALANCE","due_date":"2023-09-04","tipping_enabled":true,"computed_amount_money":{"amount":1000,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"e05d75bc-d725-487f-b568-2d7bd468c324","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"NOT_APPLICABLE"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"T57NBJNMRHX79ZV68VW8SK166R","given_name":"Ade","family_name":"Lawal","email_address":"adelawal@live.co.uk","phone_number":"+447769116015"},"invoice_number":"2023-09-03_4_T57NBJNMRHX79ZV68VW8SK166R_1","title":"Match Subs 02 Sep vs Suttonians 4s","description":"Match Subs 02 Sep vs Suttonians 4s","scheduled_at":"2023-09-04T10:00:00Z","status":"SCHEDULED","timezone":"Europ